# SETUP

In [1]:
# TechVidvan Human pose estimator
import cv2
import mediapipe as mp
import imageio,canvas
from PIL import Image,ImageSequence     
import time 
import math
import numpy as np

ModuleNotFoundError: No module named 'mediapipe'

In [2]:
# mpDraw=mp.solutions.drawing_utils
width=1280
height=720
deg = 0

In [3]:
class mpHands:
    import mediapipe as mp
    def __init__(self,maxHands=2):
        self.hands=self.mp.solutions.hands.Hands(False,maxHands)
    def Marks(self,frame):
        lmList=[]
        handsType=[]
        frameRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        result=self.hands.process(frameRGB)
        if result.multi_hand_landmarks: #if result available
        
            for hand in result.multi_handedness:
                #print(hand.classification[0])
                if hand.classification[0].label == "Left":
                    handType=0
                elif hand.classification[0].label == "Right": 
                    handType=1
                handsType.append(handType)
                
            for hand in result.multi_hand_landmarks:
                for id, lm in enumerate(hand.landmark):
                    h,w,c=frame.shape
                    coorx, coory=int(lm.x*w), int(lm.y*h)
                    lmList.append([coorx, coory])
        return lmList,handsType

class mpPose:
    import mediapipe as mp
    def __init__(self, min_detection_confidence=0.5,min_tracking_confidence=0.5):
        self.pose=self.mp.solutions.pose.Pose()
    def Marks(self,frame):
        lm_List=[]
        frameRGB=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        result=self.pose.process(frameRGB)
        if result.pose_landmarks:  # pytype: disable=attribute-error
            for id in range(32):
                h,w,c=frame.shape
                lm_List.append([int(result.pose_landmarks.landmark[id].x*w), int(result.pose_landmarks.landmark[id].y*h)])
        return lm_List


In [4]:
def position_data(lmlist):
    global wrist, thumb_tip, index_mcp, index_pip, index_tip, middle_mcp,middle_pip, middle_tip, ring_mcp, ring_tip, ring_pip, pinky_mcp, pinky_tip
    wrist = (lmlist[0][0], lmlist[0][1])
    thumb_tip = (lmlist[4][0], lmlist[4][1])
    index_mcp = (lmlist[5][0], lmlist[5][1])
    index_pip = (lmlist[6][0], lmlist[6][1])
    index_tip = (lmlist[8][0], lmlist[8][1])
    middle_mcp = (lmlist[9][0], lmlist[9][1])
    middle_pip= (lmlist[10][0], lmlist[10][1])
    middle_tip = (lmlist[12][0], lmlist[12][1])
    ring_mcp = (lmlist[13][0], lmlist[13][1])
    ring_pip = (lmlist[14][0], lmlist[14][1])
    ring_tip  = (lmlist[16][0], lmlist[16][1])
    pinky_mcp = (lmlist[17][0], lmlist[17][1])
    pinky_tip = (lmlist[20][0], lmlist[20][1])


def position_pose(lmlist):
    global right_elbow, right_wrist, left_elbow,left_wrist,right_index,left_index
    left_elbow = [lmlist[14][0], lmlist[14][1]]
    left_wrist = [lmlist[16][0], lmlist[16][1]]
    left_index = [lmlist[20][0], lmlist[20][1]]

    right_elbow = [lmlist[13][0], lmlist[13][1]]
    right_wrist = [lmlist[15][0], lmlist[15][1]]
    right_index = [lmlist[19][0], lmlist[19][1]]


def draw_line(p1, p2, size=5):
    cv2.line(frame, p1, p2, (55,175,212), size)
    # cv2.line(frame, p1, p2, (255, 255, 255), round(size / 2))

def calculate_distance(p1,p2):
    x1, y1, x2, y2 = p1[0], p1[1], p2[0], p2[1]
    lenght = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** (1.0 / 2)
    return lenght


def transparent(targetImg, x, y, size=None):
    #size chi don gian la resize foreground
    if size is not None:
        targetImg = cv2.resize(targetImg, size)
    #copy img
    newFrame = frame.copy() 

    #dung alpha channel de xac dinh mask cho foreground
    b, g, r, a = cv2.split(targetImg)
    
    
    overlay_color = cv2.merge((b, g, r))

    #filter remove noise for the circle
    mask = cv2.medianBlur(a, 1)

    h, w, _ = overlay_color.shape

    #cut the frame  
    roi = newFrame[y:y + h, x:x + w]

    frame1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
    frame2_fg = cv2.bitwise_and(overlay_color, overlay_color, mask=mask)
    newFrame[y:y + h, x:x + w] = cv2.add(frame1_bg, frame2_fg)

    return newFrame

def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img


# 1. MAGIC SHIELD

In [5]:
cam=cv2.VideoCapture(0)
cam.set(3, width)
cam.set(4, height)
cam.set(cv2.CAP_PROP_FPS, 30)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
#magic
frame_1 = cv2.imread('resources/outer_shield.png', -1)
frame_2 = cv2.imread('resources/inner_shield.png', -1)
 

while True:
    ret,frame=cam.read()
    frame=cv2.flip(frame, 1) #
    rgbframe=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result=hands.process(rgbframe)
    if result.multi_hand_landmarks: #if result available
        for hand in result.multi_hand_landmarks:
            lmList=[]
            for id, lm in enumerate(hand.landmark):
                h,w,c=frame.shape
                coorx, coory=int(lm.x*w), int(lm.y*h)
                lmList.append([coorx, coory])
                # cv2.circle(frame, (coorx, coory),6,(50,50,255), -1)
            # mpDraw.draw_landmarks(frame, hand, mpHands.HAND_CONNECTIONS)
            position_data(lmList)
            palm = calculate_distance(wrist, index_mcp)
            distance = calculate_distance(index_tip, pinky_tip)
            ratio = distance / palm
            print(ratio)
            # if (1.3>ratio>0.5):
            #     draw_line(wrist, thumb_tip)
            #     draw_line(wrist, index_tip)
            #     draw_line(wrist, middle_tip)
            #     draw_line(wrist, ring_tip)
            #     draw_line(wrist, pinky_tip)
            #     draw_line(thumb_tip, index_tip)
            #     draw_line(thumb_tip, middle_tip)
            #     draw_line(thumb_tip, ring_tip)
            #     draw_line(thumb_tip, pinky_tip)
             
            if (ratio > 1.3):
                    centerx = middle_mcp[0] 
                    centery = middle_mcp[1] #xác định tâm fg bằng tọa độ tay
                    shield_size = 3.0 #tỉ lệ của vòng đối với lòng bàn tay
                    diameter = round(palm * shield_size) #tính toán đường kính của fg
                    x1 = round(centerx - (diameter / 2)) 
                    y1 = round(centery - (diameter / 2)) #xác định góc của fg
                    # (x1,y1) là góc trên của bbox circle fg
                    h, w, c = frame.shape #lấy thông số bg
                    #căn trái
                    if x1 < 0:
                        x1 = 0
                    #
                    elif x1 > w:
                        x1 = w
                    #căn trên
                    if y1 < 0:
                        y1 = 0
                    #
                    elif y1 > h:
                        y1 = h

                    #căn phải
                    if x1 + diameter > w:
                        diameter = w - x1
                    #căn dưới
                    if y1 + diameter > h:
                        diameter = h - y1

                    
                    shield_size = diameter, diameter
                    ang_vel = 2.0
                    deg = deg + ang_vel
                    if deg > 360:
                        deg = 0
                    hei, wid, col = frame_1.shape
                    cen = (wid // 2, hei // 2)
                    M1 = cv2.getRotationMatrix2D(cen, round(deg), 1.0)
                    M2 = cv2.getRotationMatrix2D(cen, round(360 - deg), 1.0)
                    rotated1 = cv2.warpAffine(frame_1, M1, (wid, hei))
                    rotated2 = cv2.warpAffine(frame_2, M2, (wid, hei))
                    if (diameter != 0):
                        frame = transparent(rotated1, x1, y1, shield_size)
                        frame = transparent(rotated2, x1, y1, shield_size)


    # print(result)
    cv2.imshow("Image",frame)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

0.4468562533769765
0.4978913980904648
0.5110076083921433
0.4473076566210949
0.6508783828165524
0.609298898575498
0.688115925278081
2.065234458884934
1.8093478362047501
1.5046341327697115
1.6695592694976351
1.3296097289081585
1.2256602776403254
1.3905610848570997
1.2604636174607238
1.5482598520880237
1.3261910473460168
1.5499845612925356
1.2847459307967413
1.5386751988387175
1.2763110195500904
1.5214144910247798
1.270350021782735
1.5305425683832572
1.2439514084065433
1.5512435054390579
1.2908824095044478
1.5483599130628496
1.3120258732629506
1.5843927635484043
1.32407643386397
1.5995157621399705
1.362005322917603
1.6347036727067616
1.4199839742054412
1.6588968297411408
1.3766305181209268
1.66221458213829
1.3531426310810373
1.7205294320665578
1.39452428163835
1.721356654458623
1.3902845893613933
1.7238430794818833
1.3636589977734315
1.6975988801188238
1.3557107880573211
1.653602237748033
1.3595583972363934
1.6560149462416005
1.3787786590112066
1.6608574798346516
1.3824656036134984
1.6534

# 2. SLING RING PORTAL

In [12]:

vid1 = cv2.VideoCapture("resources/portal/sling_start.mp4")
vid1.set(cv2.CAP_PROP_FPS, 144)

vid2 = cv2.VideoCapture("resources/portal/portal1.mp4")
vid2.set(cv2.CAP_PROP_FPS, 144)
# Ham tinh toan toc do

def calculate_speed(startPosition, currentPosition, fps):
	global pixels_per_meter
	# Tinh toan khoang cach di chuyen theo pixel
	distance_in_pixels = math.sqrt(math.pow(currentPosition[0] - startPosition[0], 2) + math.pow(currentPosition[1] - startPosition[1], 2))
	# Tinh toan khoang cach di chuyen bang met
	#distance_in_meters = distance_in_pixels / pixels_per_meter
	# Tinh toc do met tren giay
	#speed_in_meter_per_second = distance_in_meters * fps
	return distance_in_pixels



cam=cv2.VideoCapture(0)
cam.set(3, width)
cam.set(4, height)
cam.set(cv2.CAP_PROP_FPS, 144)
findHands=mpHands(2)
findPoses=mpPose()

# Cac tham so phuc vu speed tracking
frame_idx = 0
fps = 0
# Cai dat tham so : so diem anh / 1 met, o day dang de 1 pixel = 1 met
pixels_per_meter = 1

handStartPosition = {}
handCurrentPosition = {}
global handStartPosition, handCurrentPosition
speed = [None]

signal=[0,0,0]
count1 = count2 = 0
ring = [0,0]
asd = 0
while True:
    ret,frame=cam.read()
    frame=cv2.flip(frame, 1)
    output_frame = frame.copy()
    #Bat dau xac dinh tay    
    lmList, handsType=findHands.Marks(frame)
    lm_List = findPoses.Marks(frame)
    for hand,handType in zip(lmList,handsType):
        position_data(lmList)
        position_pose(lm_List)
        #Signal left hand 
        if signal[0]==0:
            if handType == 0:     
                #tinh toan cho tay trai
                hi = calculate_distance(index_tip, middle_tip)
                hei = calculate_distance(middle_tip, wrist)
                ratio = hi / hei
                if(0.20<ratio):
                    signal[0]=1
                    print("Left hand: OK")
                    

        #Signal right hand
        if signal[0]==1:
            if calculate_distance(left_index,index_pip)<10: 
                draw_line(index_pip,middle_pip, size=50)
            if handType==1:
                 
                #tinh toan cho tay phai
                hi = calculate_distance(index_tip, middle_tip)
                hei = calculate_distance(middle_tip, wrist)
                ratio = hi / hei
                
                draw_line(right_wrist, right_wrist,size=10)
                
                if signal[1]==0:
                    if(0.20<ratio): 
                        signal[1]=1
                        print("Right hand: OK")
                #ALL SIGNAL DONE, START Moving 
                        handStartPosition = [right_wrist[0],right_wrist[1]]
                        start_time = time.time()
            

                if signal[1]==1:

                    #instant update current
                    handCurrentPosition = [right_wrist[0],right_wrist[1]]
                    
                    #end time
                    end_time = time.time()
                    if not (end_time == start_time):
                        fps = 1.0/(end_time - start_time)
                    
                    #swap value
                    [x1, y1] = handStartPosition
                    [x2, y2] = handCurrentPosition
                    handStartPosition = [x2, y2]


                    # Neu tay co di chuyen thi
                    if [x1, y1] != [x2, y2]:
                        #tinh toan toc do
                        speed = calculate_speed([x1, y1], [x2, y2],fps)
                    #cal & blending
                        #start ring
                        print(speed)
                        if 50<speed<150:
                            count1+=1
                            if count1>=4:
                                print("START")
                                ring[0]= 1         
                                count1=0

                        #open ring
                        elif speed>150:
                            count2+=1
                            print(count2)
                            if count2>=7: 
                                print("OPEN")
                                ring[1] = 1

    #vid1
    if ring[0]==1:
        r1,frame1 = vid1.read()
        if r1==True:
                rgb_frame_resize1=cv2.resize(frame1,(width,height)) 
                output_frame=cv2.addWeighted(output_frame,0.4,rgb_frame_resize1,0.6,0.0)
        
        else: 
            #vid2
            if ring[1]==1:
                r2,frame2 = vid2.read()
                if r2==True:
                    rgb_frame_resize2=cv2.resize(frame2,(width,height))
                    output_frame=cv2.addWeighted(output_frame,0.1,rgb_frame_resize2,0.9,0.0)
                else:
                    break
            else:
                vid1 = cv2.VideoCapture("resources/portal/sling_start.mp4")
                ring[0]=0
    
    cv2.imshow("result",output_frame)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

Left hand: OK
Right hand: OK
56.089214649520635
31.144823004794873
26.92582403567252
43.01162633521314
10.0
8.246211251235321
38.600518131237564
134.937022347464
119.00420160649792
115.15641536623133
START
96.60745312862771
91.59148432032315
288.5619517538652
1
549.4087731370878
2
401.29914029312346
3
27.65863337187866
107.80074211247342
119.0
START
113.0
23.021728866442675
18.867962264113206
6.324555320336759
87.20665112249179
25.0
6.708203932499369
12.36931687685298
3.605551275463989
36.05551275463989
78.85429601486528
7.0
6.082762530298219
17.0
19.6468827043885
37.48332962798263
50.00999900019995
START
67.11929677819934
7.810249675906654
6.4031242374328485
15.0
7.0
24.08318915758459
3.0
38.897300677553446
52.009614495783374
42.20189569201838
79.90619500389191
64.4437739428721
START
99.60923651951158
34.132096331752024
286.58681058276215
4
199.7248106770914
5
52.20153254455275
177.47675904185314
6
50.15974481593781
129.41792766073794
START
305.0639277266324
7
OPEN
155.12897859523216
